# Tic Tac Toe
- 3^9 = 19683 possible states
- many of them are forbidden by the game logic (e.g. all X or all O)

## 1. Step: Generate all possible states

In [69]:
symbols = ['-', 'O', 'X']
boardArray = Array{Array{Char, 1}, 1}(undef, 3^9)

for i in 1:3^9
    currentBoardStateVector = Array{Char, 1}(undef, 9)
    c = i

    for j in 1:9
        currentBoardStateVector[j] = symbols[floor(Int, (c%3+1))]
        c /= 3
    end

    boardArray[i] = currentBoardStateVector
end

println("Generated ", length(boardArray), " possible board states.")
randBoardStateId = rand(1:length(boardArray))
println("E.g. nr. ", randBoardStateId, ": ", boardArray[randBoardStateId])

Generated 19683 possible board states.
E.g. nr. 4072: ['O', 'O', 'X', '-', 'X', 'O', 'X', 'O', '-']


## 2. Step: Remove all impossible states

First remove all states which would mean one player overjumped the other one. Which means there are two more X or O on the field compared to the opponent.

In [105]:
function versionA(boardArray)
    invalidBoardStateIndexArray = Array{Int, 1}(undef, 0)
    invalidBoardStateMarkerArray = Array{Bool, 1}(undef, 3^9)
    i = 0

    for boardState in boardArray
        i += 1
        countX = 0
        countO = 0

        for ii in 1:9
            if boardState[ii] == 'X'
                countX += 1
            elseif boardState[ii] == 'O'
                countO += 1
            end
        end

        if (countX > countO+1) || (countO > countX+1)
            append!(invalidBoardStateIndexArray, i)
            invalidBoardStateMarkerArray[i] = true
        else
            invalidBoardStateMarkerArray[i] = false
        end
    end

    println("VersionA: ")
    println("> ", length(invalidBoardStateIndexArray), "/", length(boardArray), " = ", (length(invalidBoardStateIndexArray)/length(boardArray))*100, "% are invalid.")
    randIndex = rand(1:length(invalidBoardStateIndexArray))
    println("> ", "E.g. nr. ", randIndex, ": ", boardStateArray[invalidBoardStateIndexArray[randIndex]])
    return deleteat!(boardArray, invalidBoardStateMarkerArray)
end

# This version is much slower. One reason could be that it needs to count() twice for one decision.
#function versionB(boardArray)
#    validBoardArray = findall(boardState->(abs(count(i->(i=='X'),boardState) - count(i->(i=='O'),boardState)) < 2), boardArray)
#    println("VersionB: ")
#    println("> ", length(validBoardArray), "/", length(boardArray), " = ", (length(validBoardArray)/length(boardArray))*100, "% are valid.")
#    return validBoardArray
#end

validBoardArray = versionA(copy(boardArray))
#@time versionB(copy(boardArray))


VersionA: 
> 10730/19683 = 54.514047655337095% are invalid.
> E.g. nr. 5201: ['-', 'O', '-', '-', 'X', '-', 'O', 'O', 'O']


8953-element Vector{Vector{Char}}:
 ['O', '-', '-', '-', '-', '-', '-', '-', '-']
 ['X', '-', '-', '-', '-', '-', '-', '-', '-']
 ['-', 'O', '-', '-', '-', '-', '-', '-', '-']
 ['X', 'O', '-', '-', '-', '-', '-', '-', '-']
 ['-', 'X', '-', '-', '-', '-', '-', '-', '-']
 ['O', 'X', '-', '-', '-', '-', '-', '-', '-']
 ['-', '-', 'O', '-', '-', '-', '-', '-', '-']
 ['X', '-', 'O', '-', '-', '-', '-', '-', '-']
 ['X', 'O', 'O', '-', '-', '-', '-', '-', '-']
 ['-', 'X', 'O', '-', '-', '-', '-', '-', '-']
 ⋮
 ['O', '-', 'O', 'O', 'O', 'X', 'X', 'X', 'X']
 ['-', 'O', 'O', 'O', 'O', 'X', 'X', 'X', 'X']
 ['O', 'O', 'O', 'O', 'O', 'X', 'X', 'X', 'X']
 ['X', 'O', 'O', 'O', 'O', 'X', 'X', 'X', 'X']
 ['O', 'X', 'O', 'O', 'O', 'X', 'X', 'X', 'X']
 ['O', 'O', 'X', 'O', 'O', 'X', 'X', 'X', 'X']
 ['O', 'O', 'O', 'X', 'O', 'X', 'X', 'X', 'X']
 ['O', 'O', 'O', 'O', 'X', 'X', 'X', 'X', 'X']
 ['-', '-', '-', '-', '-', '-', '-', '-', '-']

# 3. Step: Classify remaining board states

All remaining board states should be classified in one of the following four classes:

1. notFinished
2. winX
3. winO
4. pad

In [127]:
# Winnings in a row
winX = filter(i->('X'==i[1]==i[2]==i[3]), validBoardArray)
append!(winX, filter(i->('X'==i[4]==i[5]==i[6]), validBoardArray))
append!(winX, filter(i->('X'==i[7]==i[8]==i[9]), validBoardArray))

# Winnings in a column
append!(winX, filter(i->('X'==i[1]==i[4]==i[7]), validBoardArray))
append!(winX, filter(i->('X'==i[2]==i[5]==i[8]), validBoardArray))
append!(winX, filter(i->('X'==i[3]==i[6]==i[9]), validBoardArray))

# Winnings in a diagonal
append!(winX, filter(i->('X'==i[1]==i[5]==i[9]), validBoardArray))
append!(winX, filter(i->('X'==i[3]==i[5]==i[7]), validBoardArray))

# Winnings in a row
winO = filter(i->('O'==i[1]==i[2]==i[3]), validBoardArray)
append!(winO, filter(i->('O'==i[4]==i[5]==i[6]), validBoardArray))
append!(winO, filter(i->('O'==i[7]==i[8]==i[9]), validBoardArray))

# Winnings in a column
append!(winO, filter(i->('O'==i[1]==i[4]==i[7]), validBoardArray))
append!(winO, filter(i->('O'==i[2]==i[5]==i[8]), validBoardArray))
append!(winO, filter(i->('O'==i[3]==i[6]==i[9]), validBoardArray))

# Winnings in a diagonal
append!(winO, filter(i->('O'==i[1]==i[5]==i[9]), validBoardArray))
append!(winO, filter(i->('O'==i[3]==i[5]==i[7]), validBoardArray))

@show length(winX)
@show length(winO)


length(winX) + length(winO) + -(length(validBoardArray)) = -6377


-6377

# 4. Step: Extract some test data

To validate the training we need some data which aren ot used for training. For this simply pick randomly X% of each class and put them out of the training data.